In [1]:
import os, time, json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [2]:
with open('C:/Users/badbr/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
    
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
# load in the df from project part 1 as basics:
import pandas as pd
basics = pd.read_csv('Data/title_basics.csv.gz')

In [5]:
# ## make a movie object using the .Movies function from tmdb
# movie = tmdb.Movies(603)

In [6]:
# ## movie objects have a .info dictionary 
# info = movie.info()

In [7]:
# movie = tmdb.Movies('tt1361336')
# info = movie.info()
# info['budget']

In [8]:
# response = movie.releases()

In [9]:
# # example from package ReadMe
# response = movie.releases()
# for c in movie.countries:
#     if c['iso_3166_1'] == 'US':
#         print(c['certification'])

In [10]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [11]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [12]:
YEARS_TO_GET = [2000,2001]

In [13]:
errors = [ ]

In [14]:
# # # Start of OUTER loop
# for YEARS in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
#     JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'
    
#     file_exists = os.path.isfile(JSON_FILE)
    
#     if file_exists == False:
    
#         with open(JSON_FILE, 'w') as f:
#             json.dump([{'imdb_id':0}],f)
        
#     df = basics.loc[basics['startYear']==YEARS].copy()
    
#     movie_ids = df['tconst'].copy()

#     previous_df = pd.read_json(JSON_FILE)
    
#     movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#     # INNER Loop
#     for movie_id in tqdm_notebook(movie_ids_to_get,
#                                   desc = f'Movies from {YEARS}',
#                                   position = 1,
#                                   leave = True):
#         try:
#             # Retrieve then data for the movie id
#             temp = get_movie_with_rating(movie_id)  
#             # Append/extend results to existing file using a pre-made function
#             write_json(temp,JSON_FILE)
#             # Short 20 ms sleep to prevent overwhelming server
#             time.sleep(0.02)

#         except Exception as e:
#             errors.append([movie_id, e])
            
#     final_year_df = pd.read_json(JSON_FILE)
#     final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEARS}.csv.gz",
#                           compression="gzip", index=False)

# print(f"- Total errors: {len(errors)}")

In [15]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',
                          position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)



    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()


    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)


    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            errors.append([movie_id, e])


    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1448 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1570 [00:00<?, ?it/s]

In [16]:
# # Start of OUTER loop
# for YEARS in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
#     JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'
#     file_exists = os.path.isfile(JSON_FILE)
    
#     if file_exists == False:
    
#         with open(JSON_FILE, 'w') as f:
#             json.dump([{'imdb_id':0}],f)
        
#     df = basics.loc[basics['startYear']==YEARS].copy()
    
#     movie_ids = df['tconst'].copy()
#     movie_ids

#     previous_df = pd.read_json(JSON_FILE)
#     previous_df
    
#     movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#     # INNER Loop
#     for movie_id in tqdm_notebook(movie_ids_to_get,
#                                   desc=f'Movies from {YEARS}',
#                                   position=1,
#                                   leave=True):
#         try:
#             # Retrieve then data for the movie id
#             temp = get_movie_with_rating(movie_id)  
#             # Append/extend results to existing file using a pre-made function
#             write_json(temp,JSON_FILE)
#             # Short 20 ms sleep to prevent overwhelming server
#             time.sleep(0.02)

#         except Exception as e:
#             continue
#     final_year_df = pd.read_json(JSON_FILE)
#     final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEARS}.csv.gz",
#                           compression="gzip", index=False)

In [17]:
# def get_movie_with_rating(movie_id):
    
#     # get the movie object for the current id 
#     movie = tmdb.Movies(movie_id)
    
#     # save the .info .releases dict 
#     info = movie.info()
    
#     releases = movie.releases()
    
#     # loop through countries in releases
#     for c in releases['countries']:
        
#         # if the country is abbre
#         if c['iso_3166_1'] == 'US':
            
#             ## save a cert key in the info dict with cert
#             info['certification'] = c['certification']
#     return info

In [18]:
# def write_json(new_data, filename):
#     """Appends a list of records (new_data) to a json file (filename). 
#     Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    
#     with open(filename, 'r+') as file:
#         #1st we load existing data in dict
#         file_data = json.load(file)
#         if (type(new_data) == lsit) & (type(file_data) == list):
#             file_data.extend(new_data)
#         else:
#             file_data.append(new_data)
#         # set files current position at offset
#         file.seek(0)
#         # convert back to json
#         json.dump(file_data, file)